In [209]:
import pandas as pd
import openpyxl as xl
from shutil import copy

Staff = pd.read_excel('Штат.xls')  # считываем файл со штатом сотрудников = Staff

Staff['full_name_dep'] = (Staff["Фамилия"] + " " + Staff["Имя"] + " " + Staff["Отчество"] + " (" + Staff["Штатная должность"]+")").astype("string") #делаем full_name - ФИО
Staff['full_name'] = (Staff["Фамилия"] + " " + Staff["Имя"] + " " + Staff["Отчество"]).astype("string")

departments = Staff['Подразделение (наименование)'].unique() #находим уникальные значения, то есть все департменты

In [210]:
# можно здесь что-то подправить, если шаблон измениться 
template_size = 50 # ppl per file (кол-во людей, которое может быть в одном файле, если окажется что в департменте работает больше, просто изменить значение )
first_row = 18     # first row of the empty list in the template

#здесь как раз и отлавливаю ошибку, если вдруг окажется что лист очень маленький, а народу в департменте много
max_people = Staff.groupby('Подразделение (наименование)')['full_name_dep'].count().max()
print(Staff['full_name_dep'])
if template_size < max_people:
    raise ValueError('The template is too small!')

last_row = first_row + template_size * 4 - 1  # последняя пустая строчка в  template
print ("===================================")
print(f'Check that the empty list in ФормаТабель.xlsx goes from row {first_row} to row {last_row}') #для дебага, и понимания того что происходит

0      Клеенкова Ольга Владимировна (Заместитель глав...
1          Панайотиди Екатерина Владимировна (Бухгалтер)
2                 Суслова Ольга Вячеславовна (Бухгалтер)
3        Толокнова Ева Александровна (Главный бухгалтер)
4               Чухляева Елена Александровна (Бухгалтер)
                             ...                        
392            Сенькина Мария Евгеньевна (Администратор)
393            Сенькина Мария Евгеньевна (Администратор)
394    Трофименко Ирина Анатольевна (Зав учебным цент...
395    Шулькин Игорь Михайлович (Зав центром во внедр...
396    Нижельская Екатерина Леонидовна (Оператор ввод...
Name: full_name_dep, Length: 397, dtype: string
Check that the empty list in ФормаТабель.xlsx goes from row 18 to row 217


In [213]:
for department in departments:

    #создаем файл для каждого департмента 
    file = f'{department}.xlsx'
    copy('ФормаТабель.xlsx', file) #название можно поменять
    wb = xl.load_workbook(file)
    sheet = wb['Табель'] #название листа, можно поменять на нужный в шаблоне на входе (то есть в ФормаТабель.xlsx )
   
    for col in [(2,28)]:
        sheet.unmerge_cells(start_row=5, start_column=col[0], end_row=5, end_column=col[1])
    #sheet.cell('P5').value = department 
    sheet['O5'] = department #добавляем название департмента в ячейку A5

    

    # добавляем всех сотрудников в файл
    people = Staff.loc[Staff["Подразделение (наименование)"] == department,                # фильтр по отделу
                       ['full_name_dep', 'Табельный номер']] # столбцы которые нам надо
    people = people.sort_values('full_name_dep')
    #print (people) check that people working w/o error
    n = people.shape[0] # n = number of people
    r = first_row       # r = current row
    
    numbering = 1 # нумерация людей в списке     
    for person in people.itertuples():
        sheet.cell(r, 3).value = person[1]   # ФИО + должность 
        sheet.cell(r, 4).value = person[2]   # Табельный номер
        sheet.cell(r, 2).value = numbering
        numbering+=1
        
        r += 4 #добавляем через две клетки нового сотрудника 
    
    # удаляем все слитые НЕИСПОЛЬЗОВАННЫЕ клетки экселя чтобы они были не создавали проблем после добавления людей в файл департмента
    #P.S. раньше происходил непонятный сдвиг линий контура, удалялись которые слова и т.п.
    
    """
    for row in range(r, last_row, 4):
        for col in [(2,2), (3,3), (4,4)]:
            sheet.unmerge_cells(start_row=row, start_column=col[0], end_row=row+1, end_column=col[3])
       
    """
    # удаляем неиспользованные строки 
    sheet.delete_rows(r, last_row + 1  - r)
    
    #корректируем print area чтобы она соотвестовала финальному результату, который нам нужен (без пустых пропусков и т.п)
    sheet.print_area = sheet.calculate_dimension()
    
    wb.save(file)    
    

    # нужен для теста (сделает один файл), удалите break если надо чтобы сделало все департаменты (весь цикл)
    #break

In [163]:
i = 2
while i<(33):
    print("(" , i , "," , i , "), ")
    i+=1

( 2 , 2 ), 
( 3 , 3 ), 
( 4 , 4 ), 
( 5 , 5 ), 
( 6 , 6 ), 
( 7 , 7 ), 
( 8 , 8 ), 
( 9 , 9 ), 
( 10 , 10 ), 
( 11 , 11 ), 
( 12 , 12 ), 
( 13 , 13 ), 
( 14 , 14 ), 
( 15 , 15 ), 
( 16 , 16 ), 
( 17 , 17 ), 
( 18 , 18 ), 
( 19 , 19 ), 
( 20 , 20 ), 
( 21 , 21 ), 
( 22 , 22 ), 
( 23 , 23 ), 
( 24 , 24 ), 
( 25 , 25 ), 
( 26 , 26 ), 
( 27 , 27 ), 
( 28 , 28 ), 
( 29 , 29 ), 
( 30 , 30 ), 
( 31 , 31 ), 
( 32 , 32 ), 
